# 3. Experimentieren

Basierend auf diesen Daten können nun weitere Experimente zur Analyse und Visualisierung dieser Daten durchgeführt werden.
Im Folgenden sind zwei Experimente implementiert:

1. Boxplot zur Darstellung der Partei-Haltungen zur Elternzeit
2. Barchart zur Darstellung der Parteien mit den meisten Abweichlern


Für beide Experimente müssen zuerst die Daten geladen werden:


In [ ]:
import json

filename = "nationalrat_raw.json"
candidates = json.load(open(filename, encoding="utf-8"))

# Elternzeit


In [ ]:
import pandas as pd # Für tabellenartige Daten
import seaborn as sns # Für die Visualisierung
import matplotlib.pyplot as plt # Für die Visualisierung

# Daten in ein Pandas DataFrame umwandeln (für die bessere Handhabung)
df = pd.DataFrame(candidates)
answer_data = []
fragen_objekte = df.head(1)['answers'][0] # Array aus allen Fragen
print(fragen_objekte)
frage = fragen_objekte[0]['text'] # Erste Frage

In [ ]:
for idx, row in df.iterrows():
    for answer in row["answers"]: # Iteriere über alle Antworten
       if answer["questionId"] == "32215": # Definiere die Frage, die du visualisieren möchtest
        answer_data.append({ # Fügt die Daten zu jedem Kandidierenden in einen Array
            "id": row["id"],
            "firstname": row["firstname"],
            "lastname": row["lastname"],
            "partyAbbreviation": row["partyAbbreviation"],
            "partyColor": row["partyColor"],
            "questionId": answer["questionId"],
            "answer": int(answer["value"])
            })
answer_df = pd.DataFrame(answer_data)
print(answer_df)

In [ ]:
# Lösche Parteien mit weniger als 75 Kandidierenden
party_counts = answer_df['partyAbbreviation'].value_counts() # Zähle die Anzahl der Kandidierenden pro Partei
parties_to_keep = party_counts[party_counts > 75].index # Definiere die Parteien, die mehr als 75 Kandidierende haben
filtered_answer_df = answer_df[answer_df['partyAbbreviation'].isin(parties_to_keep)] # Löscht die Kandidierenden, die nicht in den definierten Parteien sind

In [ ]:
plt.figure(figsize=(10, 6)) # Definiere die Größe des Plots
sns.set_theme(style="whitegrid") # Hintergrund mit Gitternetzlinien

# Parteikürzel auf der x-Achse, Zustimmung auf der y-Achse
sns.boxplot(x='partyAbbreviation', y='answer', data=filtered_answer_df, palette='Set2', showfliers=False)

# Achsenbeschriftung und Titel
plt.title(frage, fontsize=16)
plt.xlabel("Partei", fontsize=14)
plt.ylabel("Zustimmung (0-100)", fontsize=14)

# Ausgabe des Plots
plt.show()

# Abweichler


## Teil 1: Grösste Abweichler finden


In [ ]:
import pandas as pd # Für tabellenartige Daten
import numpy as np # Für numerische Operationen

df = pd.DataFrame(candidates) # Daten in ein Pandas DataFrame umwandeln (für die bessere Handhabung)

expanded_answers = []
for idx, row in df.iterrows(): # Iteriere über alle Kandidierenden
    for answer in row["answers"]: # Iteriere über alle Antworten
        expanded_answers.append({ 
            "candidate_id": row["id"],
            "firstname": row["firstname"],
            "lastname": row["lastname"],
            "partyAbbreviation": row["partyAbbreviation"],
            "questionId": answer["questionId"],
            "answer": answer["value"]
        })

answers_df = pd.DataFrame(expanded_answers)
print(answers_df)

In [ ]:
# Berechnet den Durchschnitt der Antworten für jede Partei und jede Frage
party_averages = answers_df.groupby(['partyAbbreviation', 'questionId'])['answer'].mean().reset_index() # Gruppiert die Daten nach Partei und Frage und berechnet den Durchschnitt
party_averages.rename(columns={'answer': 'party_avg'}, inplace=True) # Benennt die Spalte "answer" in "party_avg" um
print(party_averages)

In [ ]:
# Fügt den durchschnittlichen Partei-Antworten zu den individuellen Antworten hinzu
answers_with_avg = pd.merge(answers_df, party_averages, on=['partyAbbreviation', 'questionId'])

# Berechnet die Abweichung der Antwort von der durchschnittlichen Parteiantwort
answers_with_avg['deviation'] = np.abs(answers_with_avg['answer'] - answers_with_avg['party_avg']) # Berechnet die absolute Abweichung

# Berechnet die durchschnittliche Abweichung für jeden Kandidaten
candidate_deviations = answers_with_avg.groupby(['candidate_id', 'firstname', 'lastname', 'partyAbbreviation'])['deviation'].mean().reset_index() # Gruppiert die Daten nach Kandidierenden und berechnet den Durchschnitt der Abweichungen

Nun folgt noch die schöne Formatierung der Tabelle sowie die Ausgabe.


In [ ]:
# Nun Formattieren wir die Daten für die Ausgabe
# Zuerst benennen wir die Spalten auf Deutsch um
candidate_deviations.rename(columns={
    'candidate_id': 'ID',
    'firstname': 'Vorname',
    'lastname': 'Nachname',
    'partyAbbreviation': 'Partei',
    'deviation': 'Abweichung'
}, inplace=True)

candidate_deviations = candidate_deviations[candidate_deviations['Partei'] != 'Ind.']  # Die unabhängigen Kandidaten entfernen

candidate_deviations.set_index('ID', inplace=True) # Zur Formattierung der Tabelle (1. Spalte)


# Sortiert die Kandidaten nach der durchschnittlichen Abweichung (absteigend)
most_dissenting_candidates = candidate_deviations.sort_values(by='Abweichung', ascending=False)

display_df = most_dissenting_candidates.reset_index() # Nummeriert von 0 an die Resultate durch
display_df.index += 1   # Nummerierung soll bei 1 beginnen


# Ausgabe der ersten 20 Kandidierenden
print(display_df.head(20))

## Teil 2: Abweichler nach Partei


In [ ]:
import matplotlib.pyplot as plt # Für die Visualisierung
import seaborn as sns # Für die Visualisierung

def categorize_deviation(dev): # Funktion zur Kategorisierung der Abweichung
    if dev > 30:
        return "High Deviator"
    else:
        return "Aligned"

# Abweichler-Kategorisierung hinzufügen
candidate_deviations['Abweichler-Grad'] = candidate_deviations['Abweichung'].apply(categorize_deviation) # Fügt eine neue Spalte für die Kateogrisierung hinzu

# Anzahl Mitglieder pro Partei und Abweichler-Grad zählen
deviator_stats = candidate_deviations.groupby(['Partei', 'Abweichler-Grad']).size().unstack(fill_value=0)
print(deviator_stats)

In [ ]:
# Anzahl Mitglieder pro Partei hinzufügen
deviator_stats['Gesamt'] = deviator_stats.sum(axis=1) # Summiert die Anzahl der Mitglieder pro Partei (summiert die Zeile)

# Anteil der starken Abweichler berechnen
deviator_stats['% High Deviators'] = (deviator_stats['High Deviator'] / deviator_stats['Gesamt']) * 100 

# Sortiert die Parteien nach der Anzahl der Mitglieder und behält die Top 10
top_10_parties = deviator_stats.sort_values(by='Gesamt', ascending=False).head(10)

# Sortierung nach dem Prozentsatz der starken Abweichler
top_10_parties = top_10_parties.sort_values(by='% High Deviators', ascending=False)

# Erstellung des Balkendiagramms
plt.figure(figsize=(10, 6)) # Definiert die Größe des Plots
sns.barplot(x=top_10_parties.index, y=top_10_parties['% High Deviators'], palette='Set2') # Erstellt das Balkendiagramm
plt.title("Prozentsatz der starken Abweichler nach Partei (Top 10 Parteien)", fontsize=16) # Titel des Plots
plt.xlabel("") # Beschriftung der x-Achse
plt.ylabel("") # Beschriftung der y-Achse
plt.xticks(rotation=45) # Rotiert die x-Achsenbeschriftung um 45 Grad
plt.show() # Ausgabe des Plots